In [4]:
import numpy as np  # numpy!
import seaborn as sns # visualisation!
import matplotlib.pyplot as plt # visualisation!
import pandas as pd # dataframes & data analysis!

import re #for Regex

In [5]:
# Load the dataset
assets = pd.read_csv('assets.csv')
#Quick Check
pd.set_option('display.max_columns', None)  
assets.head(1)

,type,name,organization,description,created_date,url,datasheet,modality,size,sample,analysis,dependencies,included,excluded,quality_control,access,license,intended_uses,prohibited_uses,monitoring,feedback,model_card,training_emissions,training_time,training_hardware,adaptation,output_space,terms_of_service,monthly_active_users,user_distribution,failures
0,dataset,ToyMix,Mila-Quebec AI Institute,ToyMix is the smallest dataset of three extens...,2023-10-09,https://arxiv.org/pdf/2310.04292.pdf,NaN,"molecules, tasks",13B labels of quantum and biological nature.,[],Models of size 150k parameters trained on ToyM...,"['QM9', 'TOX21', 'ZINC12K']",NaN,NaN,NaN,open,CC BY-NC-SA 4.0,The datasets are intended to be used in an aca...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
assets.shape

(568, 31)

In [28]:
# Replace empty values with a specific string
non_empty_assets = assets.fillna('unknown')

In [70]:
# Function to clean and transform the dataset
def cleaning(df):
    
    # Explode the 'dependencies' column if it contains lists
    df = df.explode('dependencies')

    # List of columns to be label encoded
    id_columns = ['type', 'organization', 'description', 'created_date', 'url',
       'datasheet', 'modality', 'size', 'sample', 'analysis', 'dependencies',
       'included', 'excluded', 'quality_control', 'access', 'license',
       'intended_uses', 'prohibited_uses', 'monitoring', 'feedback',
       'model_card', 'training_emissions', 'training_time',
       'training_hardware', 'adaptation', 'output_space', 'terms_of_service',
       'monthly_active_users', 'user_distribution', 'failures']
    
    # Loop through chosen columns to label encode each column
    for col in id_columns:
        # Label encode each column by converting to categorical codes
        df[f'{col}_id'] = df[col].astype('category').cat.codes + 1
   
    # Reset the index and add 'id' column starting from 1
    df['llm_data_id'] = df.reset_index().index + 1
   
    # Return the cleaned and transformed DataFrame
    return df


clean_assets = cleaning(non_empty_assets)
clean_assets.shape

(568, 62)

In [71]:
clean_assets.columns

Index(['type', 'name', 'organization', 'description', 'created_date', 'url',
       'datasheet', 'modality', 'size', 'sample', 'analysis', 'dependencies',
       'included', 'excluded', 'quality_control', 'access', 'license',
       'intended_uses', 'prohibited_uses', 'monitoring', 'feedback',
       'model_card', 'training_emissions', 'training_time',
       'training_hardware', 'adaptation', 'output_space', 'terms_of_service',
       'monthly_active_users', 'user_distribution', 'failures', 'type_id',
       'organization_id', 'description_id', 'created_date_id', 'url_id',
       'datasheet_id', 'modality_id', 'size_id', 'sample_id', 'analysis_id',
       'dependencies_id', 'included_id', 'excluded_id', 'quality_control_id',
       'access_id', 'license_id', 'intended_uses_id', 'prohibited_uses_id',
       'monitoring_id', 'feedback_id', 'model_card_id',
       'training_emissions_id', 'training_time_id', 'training_hardware_id',
       'adaptation_id', 'output_space_id', 'terms_of_ser

In [72]:
clean_assets

,type,name,organization,description,created_date,url,datasheet,modality,size,sample,analysis,dependencies,included,excluded,quality_control,access,license,intended_uses,prohibited_uses,monitoring,feedback,model_card,training_emissions,training_time,training_hardware,adaptation,output_space,terms_of_service,monthly_active_users,user_distribution,failures,type_id,organization_id,description_id,created_date_id,url_id,datasheet_id,modality_id,size_id,sample_id,analysis_id,dependencies_id,included_id,excluded_id,quality_control_id,access_id,license_id,intended_uses_id,prohibited_uses_id,monitoring_id,feedback_id,model_card_id,training_emissions_id,training_time_id,training_hardware_id,adaptation_id,output_space_id,terms_of_service_id,monthly_active_users_id,user_distribution_id,failures_id,llm_data_id
0,dataset,ToyMix,Mila-Quebec AI Institute,ToyMix is the smallest dataset of three extens...,2023-10-09,https://arxiv.org/pdf/2310.04292.pdf,unknown,"molecules, tasks",13B labels of quantum and biological nature.,[],Models of size 150k parameters trained on ToyM...,"['QM9', 'TOX21', 'ZINC12K']",unknown,unknown,unknown,open,CC BY-NC-SA 4.0,The datasets are intended to be used in an aca...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,2,101,421,214,132,39,36,38,8,180,220,32,25,82,3,14,90,70,20,186,165,21,75,118,9,32,39,3,2,1,1
1,dataset,LargeMix,Mila-Quebec AI Institute,LargeMix is the middle-sized dataset of three ...,2023-10-09,https://arxiv.org/pdf/2310.04292.pdf,unknown,"molecules, tasks",13B labels of quantum and biological nature.,[],Models of size between 4M and 6M parameters tr...,"['L1000 VCAP', 'L1000 MCF7', 'PCBA1328', 'PCQM...",unknown,unknown,unknown,open,CC BY-NC-SA 4.0,The datasets are intended to be used in an aca...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,2,101,223,214,132,39,36,38,8,181,130,32,25,82,3,14,89,70,20,186,165,21,75,118,9,32,39,3,2,1,2
2,dataset,UltraLarge,Mila-Quebec AI Institute,UltraLarge is the largest dataset of three ext...,2023-10-09,https://arxiv.org/pdf/2310.04292.pdf,unknown,"molecules, tasks",13B labels of quantum and biological nature.,[],Models of size between 4M and 6M parameters tr...,['PM6_83M'],unknown,unknown,unknown,open,CC BY-NC-SA 4.0,The datasets are intended to be used in an aca...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,2,101,432,214,132,39,36,38,8,182,203,32,25,82,3,14,88,70,20,186,165,21,75,118,9,32,39,3,2,1,3
3,model,Lag-LLaMA,"Morgan Stanley, ServiceNow Research, Universit...",Lag-LLaMA is a general-purpose foundation mode...,2024-02-08,https://time-series-foundation-models.github.i...,unknown,text; text,unknown,unknown,Evaluated on previously unseen time series dat...,[],unknown,unknown,unknown,open,Apache 2.0,unknown,unknown,unknown,https://huggingface.co/time-series-foundation-...,https://huggingface.co/time-series-foundation-...,unknown,unknown,A single NVIDIA Tesla-P100 GPU,unknown,unknown,unknown,unknown,unknown,unknown,3,105,220,267,362,39,63,199,10,109,303,32,25,82,3,2,160,70,20,178,151,21,75,82,9,32,39,3,2,1,4
4,model,Prithvi,IBM,Prithvi is a first-of-its-kind temporal Vision...,2023-08-03,https://github.com/NASA-IMPACT/hls-foundation-os,unknown,"text, video; text, video",100M parameters (dense),unknown,unknown,['NASA HLS data'],unknown,unknown,unknown,open,Apache 2.0,unknown,unknown,unknown,https://huggingface.co/ibm-nasa-geospatial/Pri...,https://huggingface.co/ibm-nasa-geospatial/Pri...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,3,73,317,188,236,39,46,19,10,248,182,32,25,82,3,2,160,70,20,154,129,21,75,118,9,32,39,3,2,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [73]:
# create tables
llm_table = clean_assets[['llm_data_id','name', 'type_id',
                          'organization_id', 'description_id', 'created_date_id', 'url_id',
                          'datasheet_id', 'modality_id', 'size_id', 'sample_id', 'analysis_id',
                          'dependencies_id', 'included_id', 'excluded_id', 'quality_control_id',
                          'access_id', 'license_id', 'intended_uses_id', 'prohibited_uses_id',
                          'monitoring_id', 'feedback_id', 'model_card_id',
                          'training_emissions_id', 'training_time_id', 'training_hardware_id',
                          'adaptation_id', 'output_space_id', 'terms_of_service_id',
                          'monthly_active_users_id', 'user_distribution_id', 'failures_id']].drop_duplicates()

type_table = clean_assets[['type_id', 'type']].drop_duplicates().sort_values(by='type_id').reset_index(drop=True)
organization_table = clean_assets[['organization_id', 'organization']].drop_duplicates().sort_values(by='organization_id').reset_index(drop=True)
description_table = clean_assets[['description_id', 'description']].drop_duplicates().sort_values(by='description_id').reset_index(drop=True)
created_date_table = clean_assets[['created_date_id', 'created_date']].drop_duplicates().sort_values(by='created_date_id').reset_index(drop=True)
url_table = clean_assets[['url_id', 'url']].drop_duplicates().sort_values(by='url_id').reset_index(drop=True)
datasheet_table = clean_assets[['datasheet_id', 'datasheet']].drop_duplicates().sort_values(by='datasheet_id').reset_index(drop=True)
modality_table = clean_assets[['modality_id', 'modality']].drop_duplicates().sort_values(by='modality_id').reset_index(drop=True)
size_table = clean_assets[['size_id', 'size']].drop_duplicates().sort_values(by='size_id').reset_index(drop=True)
sample_table = clean_assets[['sample_id', 'sample']].drop_duplicates().sort_values(by='sample_id').reset_index(drop=True)
analysis_table = clean_assets[['analysis_id', 'analysis']].drop_duplicates().sort_values(by='analysis_id').reset_index(drop=True)
dependencies_table = clean_assets[['dependencies_id', 'dependencies']].drop_duplicates().sort_values(by='dependencies_id').reset_index(drop=True)
included_table = clean_assets[['included_id', 'included']].drop_duplicates().sort_values(by='included_id').reset_index(drop=True)
excluded_table = clean_assets[['excluded_id', 'excluded']].drop_duplicates().sort_values(by='excluded_id').reset_index(drop=True)
quality_control_table = clean_assets[['quality_control_id', 'quality_control']].drop_duplicates().sort_values(by='quality_control_id').reset_index(drop=True)
access_table = clean_assets[['access_id', 'access']].drop_duplicates().sort_values(by='access_id').reset_index(drop=True)
license_table = clean_assets[['license_id', 'license']].drop_duplicates().sort_values(by='license_id').reset_index(drop=True)
intended_uses_table = clean_assets[['intended_uses_id', 'intended_uses']].drop_duplicates().sort_values(by='intended_uses_id').reset_index(drop=True)
prohibited_uses_table = clean_assets[['prohibited_uses_id', 'prohibited_uses']].drop_duplicates().sort_values(by='prohibited_uses_id').reset_index(drop=True)
monitoring_table = clean_assets[['monitoring_id', 'monitoring']].drop_duplicates().sort_values(by='monitoring_id').reset_index(drop=True)
feedback_table = clean_assets[['feedback_id', 'feedback']].drop_duplicates().sort_values(by='feedback_id').reset_index(drop=True)
model_card_table = clean_assets[['model_card_id', 'model_card']].drop_duplicates().sort_values(by='model_card_id').reset_index(drop=True)
training_emissions_table = clean_assets[['training_emissions_id', 'training_emissions']].drop_duplicates().sort_values(by='training_emissions_id').reset_index(drop=True)
training_time_table = clean_assets[['training_time_id', 'training_time']].drop_duplicates().sort_values(by='training_time_id').reset_index(drop=True)
training_hardware_table = clean_assets[['training_hardware_id', 'training_hardware']].drop_duplicates().sort_values(by='training_hardware_id').reset_index(drop=True)
adaptation_table = clean_assets[['adaptation_id', 'adaptation']].drop_duplicates().sort_values(by='adaptation_id').reset_index(drop=True)
output_space_table = clean_assets[['output_space_id', 'output_space']].drop_duplicates().sort_values(by='output_space_id').reset_index(drop=True)
terms_of_service_table = clean_assets[['terms_of_service_id', 'terms_of_service']].drop_duplicates().sort_values(by='terms_of_service_id').reset_index(drop=True)
monthly_active_users_table = clean_assets[['monthly_active_users_id', 'monthly_active_users']].drop_duplicates().sort_values(by='monthly_active_users_id').reset_index(drop=True)
user_distribution_table = clean_assets[['user_distribution_id', 'user_distribution']].drop_duplicates().sort_values(by='user_distribution_id').reset_index(drop=True)
failures_table = clean_assets[['failures_id', 'failures']].drop_duplicates().sort_values(by='failures_id').reset_index(drop=True)


In [74]:
llm_table

,llm_data_id,name,type_id,organization_id,description_id,created_date_id,url_id,datasheet_id,modality_id,size_id,sample_id,analysis_id,dependencies_id,included_id,excluded_id,quality_control_id,access_id,license_id,intended_uses_id,prohibited_uses_id,monitoring_id,feedback_id,model_card_id,training_emissions_id,training_time_id,training_hardware_id,adaptation_id,output_space_id,terms_of_service_id,monthly_active_users_id,user_distribution_id,failures_id
0,1,ToyMix,2,101,421,214,132,39,36,38,8,180,220,32,25,82,3,14,90,70,20,186,165,21,75,118,9,32,39,3,2,1
1,2,LargeMix,2,101,223,214,132,39,36,38,8,181,130,32,25,82,3,14,89,70,20,186,165,21,75,118,9,32,39,3,2,1
2,3,UltraLarge,2,101,432,214,132,39,36,38,8,182,203,32,25,82,3,14,88,70,20,186,165,21,75,118,9,32,39,3,2,1
3,4,Lag-LLaMA,3,105,220,267,362,39,63,199,10,109,303,32,25,82,3,2,160,70,20,178,151,21,75,82,9,32,39,3,2,1
4,5,Prithvi,3,73,317,188,236,39,46,19,10,248,182,32,25,82,3,2,160,70,20,154,129,21,75,118,9,32,39,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,564,Amber,3,90,33,247,405,39,63,187,10,114,12,32,25,82,3,2,158,70,20,61,50,21,75,60,9,32,39,3,2,1
564,565,CrystalCoder,3,90,107,247,405,39,52,187,10,62,237,32,25,82,3,2,158,70,20,62,51,21,75,112,9,32,39,3,2,1
565,566,Duolingo Explain My Answer,1,57,133,138,177,39,69,199,10,248,92,32,25,82,2,48,160,70,20,186,165,21,75,118,9,32,29,3,2,1
566,567,Duolingo Max,1,57,121,138,177,39,69,199,10,248,66,32,25,82,2,48,160,70,20,186,165,21,75,118,9,32,39,3,2,1


In [75]:
# Saving each table to a CSV file
llm_table.to_csv('llm_table.csv', index=False)
type_table.to_csv('type_table.csv', index=False)
organization_table.to_csv('organization_table.csv', index=False)
created_date_table.to_csv('created_date_table.csv', index=False)
url_table.to_csv('url_table.csv', index=False)
modality_table.to_csv('modality_table.csv', index=False)
size_table.to_csv('size_table.csv', index=False)
dependencies_table.to_csv('dependencies_table.csv', index=False)
access_table.to_csv('access_table.csv', index=False)
model_card_table.to_csv('model_card_table.csv', index=False)

description_table.to_csv('description_table.csv', index=False)

datasheet_table.to_csv('datasheet_table.csv', index=False)

sample_table.to_csv('sample_table.csv', index=False)
analysis_table.to_csv('analysis_table.csv', index=False)
included_table.to_csv('included_table.csv', index=False)
excluded_table.to_csv('excluded_table.csv', index=False)
quality_control_table.to_csv('quality_control_table.csv', index=False)
license_table.to_csv('license_table.csv', index=False)
intended_uses_table.to_csv('intended_uses_table.csv', index=False)
prohibited_uses_table.to_csv('prohibited_uses_table.csv', index=False)
monitoring_table.to_csv('monitoring_table.csv', index=False)
feedback_table.to_csv('feedback_table.csv', index=False)
training_emissions_table.to_csv('training_emissions_table.csv', index=False)
training_time_table.to_csv('training_time_table.csv', index=False)
training_hardware_table.to_csv('training_hardware_table.csv', index=False)
adaptation_table.to_csv('adaptation_table.csv', index=False)
output_space_table.to_csv('output_space_table.csv', index=False)
terms_of_service_table.to_csv('terms_of_service_table.csv', index=False)
monthly_active_users_table.to_csv('monthly_active_users_table.csv', index=False)
user_distribution_table.to_csv('user_distribution_table.csv', index=False)
failures_table.to_csv('failures_table.csv', index=False)

In [ ]:
# add llm ids to dependecies if they are in db